In [1]:
#  相關套件

import time
import random
import json
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
from collections import deque
from concurrent.futures import ThreadPoolExecutor
import urllib.parse

WEB_NAME = "yes123_人力銀行"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 Safari/537.36",
    "Referer": "https://www.yes123.com.tw",
}

print(f"開始執行 {WEB_NAME} ")

開始執行 yes123_人力銀行 


In [ ]:
## 取得網站所有職業總覽
# 1. 取得 JSON 資料
# jobcat 檔案名稱

file_jobcat_json = f"{WEB_NAME}_jobcat_json.txt"
url_JobCat = "https://www.yes123.com.tw/json_file/work_mode.json"

response_jobcat = requests.get(url_JobCat, headers=HEADERS, timeout=10)
response_jobcat.raise_for_status()
jobcat_data = response_jobcat.json()["listObj"]
with open(file_jobcat_json, "w", encoding="utf-8") as f:
    json.dump(jobcat_data, f, ensure_ascii=False, indent=4)
print(f"職業總覽資料已儲存為 {file_jobcat_json}")


# 2. 定義遞迴函式展平資料
def flatten_jobcat_recursive(node_list, parent_des=None, parent_no=None):
    flat_list = []
    for level_1_node in node_list:
        parent_name = level_1_node.get("level_1_name")

        # Check if the inner list exists and is not empty
        if "list_2" in level_1_node and level_1_node["list_2"]:
            # Loop through the level 2 categories within this parent
            for level_2_node in level_1_node["list_2"]:
                row = {
                    "level_1_name": parent_name,
                    "level_2_code": level_2_node.get("code"),
                    "level_2_name": level_2_node.get("level_2_name"),
                }
                flat_list.append(row)
    return flat_list


# 3. 執行結果轉為 DataFrame
flattened_data = flatten_jobcat_recursive(jobcat_data)
df_jobcat = pd.json_normalize(flattened_data)
df_jobcat.to_excel(f"{WEB_NAME}_category.xlsx", index=False)
print(f"職業總覽資料已轉換為 '{WEB_NAME}_category.xlsx'")

# 篩選出 IT 相關的工作
mask = df_jobcat["level_2_code"].astype(str).str.startswith("2_1011")
df_it_jobs = df_jobcat[mask]
df_it_jobs.head(5)

In [3]:
import urllib.parse

# 產生 yes123 人力銀行網址 https://www.yes123.com.tw 根據提供的 (關鍵字和職缺類別) 轉換為職缺網址


def catch_yes123_url(KEYWORDS, CATEGORY, ORDER="date", PAGE_NUM=1):
    """
    這個函數會根據給定的關鍵字、類別、排序和頁碼參數，
    構建一個 yes123 求職網的完整職缺網址。

    參數:
    KEYWORDS (str): 職缺的關鍵字，例如 "雲端工程師"。若無則傳入空字串 ""。
    CATEGORY (str or list): 職缺的類別代碼，例如 "2_1011_0001_0000" 或者類別代碼的列表。
                            若無則傳入空字串 ""。
    ORDER (str, optional): 排序方式。可選值為 "relevance" (相關性) 或 "date" (最新日期)。
                           預設為 "date"。
    PAGE_NUM (int, optional): 指定的頁碼。預設為 1。

    返回:
    str: 生成的 yes123 職缺網址。
    """
    BASE_URL = "https://www.yes123.com.tw/wk_index/joblist.asp"

    # 確保頁碼至少為 1，避免負數或 0 造成計算錯誤
    safe_page_num = max(1, PAGE_NUM)
    # 根據頁碼計算 strrec 的值 (每頁 30 筆)
    strrec_value = (safe_page_num - 1) * 30

    # 建立一個參數字典來儲存所有查詢參數
    params = {"strrec": strrec_value, "search_type": "job", "search_from": "joblist"}

    # 根據排序參數設定 order_by 和 order_ascend
    if ORDER == "date":
        params["order_by"] = "m_date"
        params["order_ascend"] = "desc"
    else:  # 預設為相關性排序 (relevance)
        params["order_by"] = "neworder"
        params["order_ascend"] = "asc"

    # 如果有提供關鍵字，加入必要的 search_key_word 參數
    if KEYWORDS:
        params["search_key_word"] = KEYWORDS

    # 如果有提供職務類別，加入到參數中
    if CATEGORY:
        if isinstance(CATEGORY, list):
            # 如果是列表，將其轉換為字串，並用逗號分隔
            params["find_work_mode1"] = ",".join(CATEGORY)
        else:
            params["find_work_mode1"] = CATEGORY

    # 使用 urllib.parse.urlencode 將字典轉換為查詢字串，它會自動處理 & 和編碼
    query_string = urllib.parse.urlencode(params)

    return f"{BASE_URL}?{query_string}"



# # --- 測試範例 ---

# KEYWORDS_STR = "雲端工程師"
# CATEGORY_CODE1 = "2_1011_0001_0000"  # 單一類別
# CATEGORY_CODE2 = ["2_1011_0001_0000", "2_1011_0002_0000"]  # 多個類別

# # 1. 有關鍵字, 單一類別, 相關性排序, 第 1 頁
# print("1. 有關鍵字, 單一類別, 相關性排序, 第 1 頁:")
# url_1 = catch_yes123_url(
#     KEYWORDS_STR, CATEGORY_CODE1, ORDER="relevance"
# )  # PAGE_NUM 省略，預設為 1
# print(url_1, "\n")

# # 2. 無關鍵字, 多個類別, 最新日期排序, 第 2 頁
# print("2. 無關鍵字, 多個類別, 最新日期排序, 第 2 頁:")
# url_2 = catch_yes123_url("", CATEGORY_CODE2, ORDER="date", PAGE_NUM=2)
# print(url_2, "\n")

# # 3. 有關鍵字, 無類別, 最新日期排序, 第 3 頁
# print("3. 有關鍵字, 無類別, 最新日期排序, 第 3 頁:")
# url_3 = catch_yes123_url(KEYWORDS_STR, "", ORDER="date", PAGE_NUM=3)
# print(url_3, "\n")

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from tqdm import tqdm  # 引入 tqdm
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def fetch_yes123_job_urls(
    initial_url: str, max_pages: int = None, timeout: int = 15
) -> list[str]:
    """
    從 yes123 網站擷取指定搜尋條件下的所有職缺網址。

    此函式優化了抓取流程：
    1. 只請求一次第一頁，同時用於獲取總頁數和解析第一頁的職缺。
    2. 為網路請求和頁面解析增加了錯誤處理，提升程式健壯性。
    3. 動態拼接 URL，移除了硬編碼的 BASE_URL。

    Args:
        initial_url (str): 搜尋結果的第一頁網址。
        max_pages (int, optional): 欲抓取的最大頁數。如果為 None，則自動抓取所有頁面。預設為 None。
        timeout (int, optional): 請求的超時秒數。預設為 15。

    Returns:
        list[str]: 包含所有不重複的工作職缺網址的列表。

    Raises:
        requests.exceptions.RequestException: 當初次請求失敗時拋出。
    """
    # 初始化一個空列表來儲存 URL
    job_url_list = []

    # 發送請求並解析 HTML
    response = requests.get(initial_url, headers=HEADERS, verify=False, timeout=timeout)
    response.raise_for_status()  # 若請求失敗，拋出例外
    soup = BeautifulSoup(response.text, "html.parser")

    # 解析職缺連結，並逐一 append 到列表中
    link_tags = soup.select(".Job_opening_M > a.Job_opening_block")
    for tag in link_tags:
        if "href" in tag.attrs:
            full_url = urljoin(initial_url, tag["href"])
            job_url_list.append(full_url)

    # 獲取最大頁碼
    options = soup.select("#inputState option")
    max_total_pages = max(
        int(option["value"]) for option in options if option["value"].isdigit()
    )

    # 如果指定了最大頁數，則使用該頁數，否則使用實際的最大頁數
    pages_to_fetch = min(max_total_pages, max_pages) if max_pages else max_total_pages

    # 解析指定的頁數，並顯示進度條
    for page in tqdm(range(2, pages_to_fetch + 1), desc="讀取頁面網址", unit="頁"):
        next_page_url = f"{initial_url}&strrec={(page - 1) * 30}"
        response = requests.get(next_page_url, headers=HEADERS, verify=False, timeout=timeout)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")

        link_tags = soup.select(".Job_opening_M > a.Job_opening_block")
        for tag in link_tags:
            if "href" in tag.attrs:
                full_url = urljoin(initial_url, tag["href"])
                job_url_list.append(full_url)

    return job_url_list


# # 測試範例
# initial_url = "https://www.yes123.com.tw/wk_index/joblist.asp?strrec=0&search_type=job&search_from=joblist&order_by=neworder&order_ascend=asc&search_key_word=%E9%9B%B2%E7%AB%AF%E5%B7%A5%E7%A8%8B%E5%B8%AB&find_work_mode1=2_1011_0001_0000"

# # 指定最大頁數
# specified_max_pages = 2  # 例如，想抓取前 2 頁

# job_urls = fetch_yes123_job_urls(initial_url, max_pages=specified_max_pages)
# print(f"總共筆數 : {len(job_urls)}")

In [5]:
# 從指定的職缺網址獲取職缺的相關數據


def fetch_yes123_job_data(job_url):
    """
    從指定的 yes123 職缺網址獲取詳細資訊，並依照預設順序整理。

    Args:
        job_url (str): 職缺的網址。

    Returns:
        dict: 包含職缺詳細資訊的字典，若頁面不存在或已關閉則返回 None。
    """

    try:
        response = requests.get(job_url, headers=HEADERS, timeout=15)
        response.raise_for_status()

        if "此工作機會已關閉" in response.text or "您要找的頁面不存在" in response.text:
            return None

        soup = BeautifulSoup(response.text, "html.parser")
        scraped_data = {"職缺網址": job_url}
        header_block = soup.select_one("div.box_job_header_center")
        if header_block:
            title_tag = header_block.select_one("h1")
            scraped_data["職缺名稱"] = (
                title_tag.get_text(strip=True) if title_tag else "N/A"
            )

            company_tag = header_block.select_one("a.link_text_black")
            scraped_data["公司名稱"] = (
                company_tag.get_text(strip=True) if company_tag else "N/A"
            )

        # --- 遍歷所有資訊區塊 (class="job_explain") ---
        all_sections = soup.select("div.job_explain")
        for section in all_sections:
            section_title_tag = section.select_one("h3")
            if not section_title_tag:
                continue

            section_title = section_title_tag.get_text(strip=True)

            # Key-Value 結構的區塊
            if section_title in ["徵才說明", "工作條件", "企業福利", "技能與求職專長"]:
                list_items = section.select("ul > li")
                for item in list_items:
                    key_tag = item.select_one("span.left_title")
                    value_tag = item.select_one("span.right_main")
                    if key_tag and value_tag:
                        key = key_tag.get_text(strip=True).replace("：", "")
                        value = value_tag.get_text(strip=True, separator="\n")
                        if key in scraped_data:  # 處理重複的 key (例如休假制度)
                            scraped_data[key] += f"\n(補充) {value}"
                        else:
                            scraped_data[key] = value

            # 純文字列表區塊 (法定保障)
            elif section_title == "法定保障":
                items = section.select("li > span.exception")
                scraped_data[section_title] = (
                    ", ".join([i.get_text(strip=True) for i in items])
                    if items
                    else "N/A"
                )

            # 純文字描述區塊 (其他條件)
            elif section_title == "其他條件":
                item = section.select_one("li > span.exception")
                scraped_data[section_title] = (
                    item.get_text(strip=True, separator="\n") if item else "N/A"
                )

            # 標籤結構區塊 (徵才特色)
            elif section_title == "徵才特色":
                tags = section.select("span.recruit_features")
                scraped_data[section_title] = (
                    ", ".join([tag.get_text(strip=True) for tag in tags])
                    if tags
                    else "N/A"
                )

            # 應徵方式區塊 (需獨立處理，因其結構不同)
            elif section_title == "應徵方式":
                contact_item = section.select_one("ul > li")
                if contact_item and "連絡人" in contact_item.get_text():
                    key_tag = contact_item.select_one("span.left_title")
                    value_tag = contact_item.select_one("span.right_main")
                    if key_tag and value_tag:
                        key = key_tag.get_text(strip=True).replace("：", "")
                        scraped_data[key] = value_tag.get_text(strip=True)

        # ---  根據預設順序，建立一個新的、排序好的字典 ---

        DESIRED_ORDER = [
            "職缺網址",
            "職缺名稱",
            "公司名稱",
            "工作內容",
            "薪資待遇",
            "休假制度",
            "上班日期",
            "上班時段",
            "工作性質",
            "工作地點",
            "職務類別",
            "需求人數",
            "管理人數",
            "出差說明",
            "徵才特色",
            "學歷要求",
            "科系要求",
            "工作經驗",
            "身份類別",
            "法定保障",
            "保險福利",
            "獎金制度",
            "輔助津貼",
            "休閒娛樂",
            "福利設施",
            "其他福利",
            "更多福利",
            "電腦技能",
            "其他條件",
            "連絡人",
        ]

        ordered_details = {}
        for key in DESIRED_ORDER:
            # 如果暫存字典裡有這個 key，就加到新的有序字典裡
            if key in scraped_data:
                ordered_details[key] = scraped_data[key]

        # 為了保險起見，如果爬到了未在樣板中的新欄位，也把它們加到最後面
        for key, value in scraped_data.items():
            if key not in ordered_details:
                ordered_details[key] = value

        # df = pd.json_normalize(ordered_details)
        return ordered_details

    except Exception as e:
        print(f"處理過程中發生未知錯誤: {e}")
        return None


# # 測試範例
# job_url_to_scrape = "https://www.yes123.com.tw/wk_index/job.asp?p_id=1518627_23225023&job_id=20220120031002_8984866"
# job_data = fetch_yes123_job_data(job_url_to_scrape)
# pd.json_normalize(job_data)

In [6]:
# 根據關鍵字與職業類別 獲取所有工作職位的資料

import time
import logging
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import List, Dict, Optional, Any

SEARCH_TIMESTAMP = time.strftime("%Y-%m-%d", time.localtime(time.time()))
JOBCAT_CODE = "2_1011_0001_0000"
KEYWORDS = "雲端工程師"
FILE_NAME = f"({SEARCH_TIMESTAMP})_{WEB_NAME}_{KEYWORDS}_{JOBCAT_CODE}"
MAX_WORKERS = 10  # 同時運行的執行緒數量

print(f"開始執行 {FILE_NAME}")
cata_url = catch_yes123_url(KEYWORDS, JOBCAT_CODE, ORDER="date")
job_urls = fetch_yes123_job_urls(cata_url)


job_data_list = []
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    # 建立 future 物件對應每個 URL 的爬取任務
    future_to_url = {
        executor.submit(fetch_yes123_job_data, url): url for url in job_urls
    }

    # 使用 tqdm 顯示進度條
    progress_bar = tqdm(
        as_completed(future_to_url), total=len(job_urls), desc="獲取yes123職缺資料"
    )
    for future in progress_bar:
        result = future.result()
        if result:  # 只有在成功爬取到資料時才加入列表
            job_data_list.append(result)


all_jobs_df = pd.DataFrame(job_data_list)

print(all_jobs_df.shape)

all_jobs_df.head(1)

開始執行 (2025-06-17)_yes123_人力銀行_雲端工程師_2_1011_0001_0000


獲取yes123職缺資料: 100%|██████████| 570/570 [00:46<00:00, 12.13it/s]

(570, 39)


,職缺網址,職缺名稱,公司名稱,工作內容,薪資待遇,休假制度,上班時段,工作性質,工作地點,職務類別,...,輔助津貼,休閒娛樂,其他福利,福利設施,更多福利,其他條件,交通工具,外語能力,上班制度,取得認證
0,https://www.yes123.com.tw/wk_index/job.asp?p_i...,韌體工程師-(苗栗),浩誠科技有限公司,1.﻿﻿撰寫與設計韌體程式\n2.﻿﻿與客戶 討論規格，並執行韌體產品的開發流程\n3.協助...,薪資面議(經常性薪資達4萬元含以上),週休二日,依公司規定、白天班,全職,苗栗縣苗栗市南勢里八甲192之2號2樓,韌體工程師,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# all_jobs_df.to_csv (f"{FILE_NAME}.csv", index=False, encoding='utf-8-sig')
# print (f"已將所有職缺資料儲存到 {FILE_NAME}.csv")

all_jobs_df.to_excel(f"{FILE_NAME}.xlsx", index=False)
print(f"已將所有職缺資料儲存到 {FILE_NAME}.xlsx")

已將所有職缺資料儲存到 (2025-06-17)_yes123_人力銀行_雲端工程師_2_1011_0001_0000.xlsx


In [20]:
df_columns = pd.DataFrame(all_jobs_df.columns, columns=["欄位名稱"])
df_columns.insert(0, "序號", range(1, len(df_columns) + 1))
df_columns

,序號,欄位名稱
0,1,職缺網址
1,2,職缺名稱
2,3,公司名稱
3,4,工作內容
4,5,薪資待遇
5,6,休假制度
6,7,上班時段
7,8,工作性質
8,9,工作地點
9,10,職務類別
